In [2]:
import { load } from "dotenv"
const env = await load({
  envPath: ".env.local",
})

const process = { env }

const chatOptions = {
  apiKey: process.env.Tongyi_API_KEY,
  baseURL: process.env.BASE_URL
}

In [3]:
import OpenAI from "openai"

const openai = new OpenAI(chatOptions)

function getCurrentWeather({ location, unit="fahrenheit"}){
  const  weather_info = {
       "location": location,
       "temperature": "72",
       "unit": unit,
       "forecast": ["sunny", "windy"],
   }
   return JSON.stringify(weather_info);
}

const tools = [
  {
    type: "function",
    function: {
      name: "getCurrentWeather",
      description: "Get the current weather in a given location",
      parameters: {
        type: "object",
        properties: {
          location: {
            type: "string",
            description: "The city and state, e.g. San Francisco, CA",
          },
          unit: { type: "string", enum: ["celsius", "fahrenheit"] },
        },
        required: ["location"],
      },
    },
  }
]

In [4]:
const messages = [
  {
    role: "user",
    content: "北京的天气怎么样",
  },
]

const result = await openai.chat.completions.create({
  messages,
  model: "qwen-plus",
  tools,
})
console.log(result.choices[0])


{
  message: {
    content: "",
    role: "assistant",
    tool_calls: [
      {
        function: { name: "getCurrentWeather", arguments: '{"location": "北京"}' },
        index: 0,
        id: "call_4b7ad6a353ff4edabf7351",
        type: "function"
      }
    ]
  },
  finish_reason: "tool_calls",
  index: 0,
  logprobs: null
}


禁用函数调用

In [5]:
const messages = [
  {
    role: "user",
    content: "北京的天气怎么样",
  },
]

const result = await openai.chat.completions.create({
  messages,
  model: "qwen-plus",
  tools,
  tool_choice: "none"
})
console.log(result.choices[0])

{
  message: {
    content: "我无法提供实时信息。您可以通过手机上的天气应用程序或搜索引擎查询北京的最新天气情况。",
    role: "assistant"
  },
  finish_reason: "stop",
  index: 0,
  logprobs: null
}


强制函数调用

In [ ]:
const messages = [
  {
    role: "user",
    content: "你好",
  },
]

const result = await openai.chat.completions.create({
  messages,
  model: "qwen-plus",
  tools,
  tool_choice: {
    type: "function",
    function: {
      name: "getCurrentWeather",
    },
  },
})
console.log(result.choices[0])


{
  message: {
    content: "",
    role: "assistant",
    tool_calls: [
      {
        function: { name: "getCurrentWeather", arguments: '{"location": "北京"}' },
        index: 0,
        id: "call_f7efa4befd1b450f88aaa7",
        type: "function"
      }
    ]
  },
  finish_reason: "tool_calls",
  index: 0,
  logprobs: null
}


In [5]:
const functions = {
  getCurrentWeather: getCurrentWeather,
}

const functionInfo = result.choices[0].message.tool_calls[0].function
const functionName = functionInfo.name
const functionParams = functionInfo.arguments

const functionResult = functions[functionName](functionParams)

console.log(functionResult)

{"temperature":"72","unit":"fahrenheit","forecast":["sunny","windy"]}


并发调用函数

In [6]:
function getCurrentTime({format = "ios"}={}){
  let currentTime;
  switch(format){
    case "ios":
      currentTime = new Date().toISOString()
      break;
    case "locale":
      currentTime = new Date().toLocaleString()
      break;
    default:
      currentTime = new Date().toString()
      break;
  }
  return currentTime;
}

In [7]:
const tools = [
  {
    type: "function",
    function: {
      name: "getCurrentTime",
      description: "Get the current time in a given format",
      parameters: {
        type: "object",
        properties: {
          format: {
            type: "string",
            enum: ["iso", "locale", "string"],
            description: "The format of the time, e.g. iso, locale, string",
          },
        },
        required: ["format"],
      },
    },
  },
  {
    type: "function",
    function: {
      name: "getCurrentWeather",
      description: "Get the current weather in a given location",
      parameters: {
        type: "object",
        properties: {
          location: {
            type: "string",
            description: "The city and state, e.g. San Francisco, CA",
          },
          unit: { type: "string", enum: ["celsius", "fahrenheit"] },
        },
        required: ["location", "unit"],
      },
    },
  },
]


In [12]:
const messages = [
  {
      "role": "user",
      "content": "请帮我查询当前的时间, 然后帮我查询北京的天气"
  }
]

const result = await openai.chat.completions.create({
  messages,
  model: "qwen-plus",
  tools,
})

console.log(result.choices[0])

{
  message: {
    content: "",
    role: "assistant",
    tool_calls: [
      {
        function: { name: "getCurrentTime", arguments: '{"format": "iso"}' },
        index: 0,
        id: "call_ea7164f785924bed9c1da7",
        type: "function"
      }
    ]
  },
  finish_reason: "tool_calls",
  index: 0,
  logprobs: null
}


理想输出

```json
message: {
    role: "assistant",
    content: null,
    tool_calls: [
      {
        id: "xxxx",
        type: "function",
        function: {
          name: "getCurrentWeather",
          arguments: '{\n  "location": "Beijing",\n  "unit": "celsius"\n}'
        }
      },
      {
        id: "xxx",
        type: "function",
        function: {
          name: "getCurrentTime",
          arguments: '{\n  "format": "locale"\n}'
        }
      }
    ]
  },
```

特性不稳定，跑不出来 :->
